In [1]:
import torch
from torch import nn
import torchvision as tv
import time

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
def train_model(**kwargs) -> None:
    loss_ = kwargs['loss']
    trainer_ = kwargs['trainer']
    num_epochs_ = kwargs['num_epochs']
    model_ = kwargs['model']
    train_ = kwargs['train']
    test_ = kwargs['test']
    device_ = kwargs['device']
    name = kwargs.get('name')
    
    model_ = model_.to(device_)

    for epoch in range(num_epochs_):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        n = 0
        start=time.time()
        
        model_.train()
        for i, (X, y)  in enumerate(train_):
            X, y = X.to(device_), y.to(device_)
            trainer_.zero_grad()
            
            if name == 'inception_v3':
                y_pred, _ = model_(X)
            else:
                y_pred = model_(X)
                
            l = loss_(y_pred, y)
            l.backward()
            trainer_.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
            n += y.shape[0]

            if i % 50 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc / n:.3f}. Train Loss: {train_loss / n:.3f}")
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        nt = 0
        model_.eval()
        for X, y in test_:
            X, y = X.to(device_), y.to(device_)
            
            if name == 'inception_v3':
                y_pred, _ = model_(X)
            else:
                y_pred = model_(X)
                
            l = loss_(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            nt += y.shape[0]
        
        print(f'epoch {epoch + 1}, loss {train_loss / n:.4f}, train acc {train_acc / n:.3f}, test acc {test_acc / nt:.3f}, time {time.time() - start:.1f} sec')
    torch.cuda.empty_cache()

In [ ]:
BATCH_SIZE = 256

transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('./DSDZ4', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('./DSDZ4', train=False, transform=transoforms, download=True, split='mnist')

train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
train_dataset.targets.shape

In [ ]:
model = tv.models.resnet18(pretrained=True)

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Linear(in_features=512, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        
trainer = torch.optim.Adam(params_to_update, lr=0.001)

train_loss_resnet18 = train_model(
    loss=nn.CrossEntropyLoss(reduction='sum'),
    trainer=trainer,
    num_epochs=1,
    model=model,
    train=train_iter,
    test=test_iter,
    device=device
)

# Step 0. time since epoch: 2.737. Train acc: 0.126. Train Loss: 2.348
# Step 50. time since epoch: 130.843. Train acc: 0.735. Train Loss: 1.162
# epoch 1, loss 1.0837, train acc 0.756, test acc 0.713, time 174.2 sec

In [ ]:
model = tv.models.densenet161(pretrained=True)

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Linear(in_features=2208, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        
trainer = torch.optim.Adam(params_to_update, lr=0.001)

train_loss_densenet161 = train_model(
    loss=nn.CrossEntropyLoss(reduction='sum'),
    trainer=trainer,
    num_epochs=1,
    model=model,
    train=train_iter,
    test=test_iter,
    device=device
)

# Step 0. time since epoch: 2.367. Train acc: 0.164. Train Loss: 2.318
# Step 50. time since epoch: 117.742. Train acc: 0.726. Train Loss: 1.206
# Step 100. time since epoch: 234.608. Train acc: 0.808. Train Loss: 0.867
# Step 150. time since epoch: 350.229. Train acc: 0.847. Train Loss: 0.704
# Step 200. time since epoch: 468.621. Train acc: 0.867. Train Loss: 0.606
# epoch 1, loss 0.5601, train acc 0.877, test acc 0.943, time 629.4 sec

In [ ]:
model = tv.models.vgg16(pretrained=True)

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
    
model.classifier[6] = nn.Linear(in_features=4096, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
trainer = torch.optim.Adam(params_to_update, lr=0.001)

train_loss_vgg16 = train_model(
    loss=nn.CrossEntropyLoss(reduction='sum'),
    trainer=trainer,
    num_epochs=1,
    model=model,
    train=train_iter,
    test=test_iter,
    device=device
)

# Step 0. time since epoch: 1.826. Train acc: 0.090. Train Loss: 2.396
# Step 50. time since epoch: 82.464. Train acc: 0.692. Train Loss: 1.101
# Step 100. time since epoch: 163.070. Train acc: 0.765. Train Loss: 0.838
# Step 150. time since epoch: 243.769. Train acc: 0.796. Train Loss: 0.719
# Step 200. time since epoch: 324.404. Train acc: 0.814. Train Loss: 0.648
# epoch 1, loss 0.6129, train acc 0.824, test acc 0.933, time 440.1 sec

In [ ]:
BATCH_SIZE = 256
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('./DSDZ4', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('./DSDZ4', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

model = tv.models.inception_v3(pretrained=True)

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(in_features=2048, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
trainer = torch.optim.Adam(params_to_update, lr=0.001)

train_loss_inception_v3 = train_model(
    loss=nn.CrossEntropyLoss(reduction='sum'),
    trainer=trainer,
    num_epochs=1,
    model=model,
    train=train_iter,
    test=test_iter,
    device=device,
    name='inception_v3'
)

# Step 0. time since epoch: 1.679. Train acc: 0.078. Train Loss: 2.349
# Step 50. time since epoch: 88.405. Train acc: 0.584. Train Loss: 1.525
# Step 100. time since epoch: 174.357. Train acc: 0.676. Train Loss: 1.209
# Step 150. time since epoch: 260.755. Train acc: 0.721. Train Loss: 1.041
# Step 200. time since epoch: 346.913. Train acc: 0.746. Train Loss: 0.939
# epoch 1, loss 0.8865, train acc 0.764, test acc 0.000, time 422.5 sec

In [17]:
import pandas as pd

df = pd.DataFrame([[1.0837, 0.5601, 0.6129, 0.8865]], columns=['resnet18', 'densenet161', 'vgg16', 'inception_v3'])
df

,resnet18,densenet161,vgg16,inception_v3
0,1.0837,0.5601,0.6129,0.8865


In [16]:
torch.cuda.empty_cache()